## Langchain 라이브러리의 활용

### LangChain(2022.10)
- 자연어처리(NLP)와 사전 학습된 LLM 모델을 사용하여 다양한 응용프로그램을 구축할 수 있게 해주는 라이브러리(설계 및 배포 기능까지 지원함)
- 데이터베이스, API, 외부 서비스 등과의 통합을 지원하는 인터페이스 제공(openai외에도 Anthropic, Hugging Face, Google 등 다양한 기업들의 LLM과 연동됨)
- RAG 구현을 위한 기본적인 인터페이스와 임베딩, Advanced RAG 등의 기능을 제공
- ALMessage, UserMesage, SystemMessage 등의 클래스로 대화 메시지를 처리하고 관리함
- 공식 페이지 : https://www.langchain.com/
- langchain docs 페이지 : https://python.langchain.com/v0.2/docs/introduction/

In [22]:
!pip install langchain

In [25]:
#Langchain 과 openai의 GPT 모델을 결합한 기능을 제공
!pip install langchain-openai

In [26]:
#Langchian 생태계를 확장하고 기본 기능을 보완하는 다양한 도구와의 통합을 제공
!pip install langchain_community 

### 모델 로드 및 질의

In [27]:
#ChatOpenAI : Langchain과 통합되어있는 openAI의 대화용 LLM을 불러오는 함수
from langchain_openai import ChatOpenAI
from getpass import getpass

In [12]:
MY_API_KEY = getpass('OpenAI API Key:')

OpenAI API Key: ········


In [13]:
chat_model= ChatOpenAI(model = 'gpt-3.5-turbo',
                       api_key= MY_API_KEY
                      )
chat_model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000026F6711FB60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026F6714D400>, root_client=<openai.OpenAI object at 0x0000026F66C5D190>, root_async_client=<openai.AsyncOpenAI object at 0x0000026F6711FB90>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [14]:
type(chat_model)

langchain_openai.chat_models.base.ChatOpenAI

In [17]:
question = '도전적인 스포츠는?'

#invoke:langchain에서 언어 모델의 응답(추론) 결과를 반환해주는 함수
answer= chat_model.invoke(question)
answer

AIMessage(content='등산, 스카이다이빙, 스카이런닝, 서핑, 랄리경주, 스노우보드 등의 스포츠는 도전적이고 흥미로운 활동입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 20, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dc5cc14e-8d28-4913-baa8-b3f481c6c751-0', usage_metadata={'input_tokens': 20, 'output_tokens': 71, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
print('질의:', question)
print('응답:', answer.content)

질의: 도전적인 스포츠는?
응답: 등산, 스카이다이빙, 스카이런닝, 서핑, 랄리경주, 스노우보드 등의 스포츠는 도전적이고 흥미로운 활동입니다.


## 프롬프트 탬플릿 활용
- 규격화된 탬플릿을 활용하여 좀 더 체계적인 구성으로 질의가 가능
- 모듈화, 코드 재사용성, 일관성(버전관리 및 테스트) 향상 

In [29]:
from langchain.prompts import PromptTemplate

In [32]:
#from_template: 문자열 포매팅을 기본적으로 활용하여 탬플릿을 설정해주는 함수
pt= PromptTemplate.from_template('{context}중에서 {topic}은/는?')

prompt = pt.format(context='직업',
                   topic='도전적인 직업'
                  )

print(prompt)

직업중에서 도전적인 직업은/는?


In [38]:
# PromptTemplat을 활용하여 질의에 맞는 응답을 생성하고 출력하는 사용자 정의 함수 선언


def get_answer(context, topic):
    pt= PromptTemplate.from_template('{context}중에서 {topic}은/는?')
    prompt = pt.format(context= context,
                   topic=topic
                  )
    #위에서 작성한 탬플릿으로 모델에 질의
    answer= chat_model.invoke(prompt)
    return {'질의':prompt, '응답': answer.content.strip()}



In [42]:
print(get_answer('직업', '육체적으로 힘든 직업'))

{'질의': '직업중에서 육체적으로 힘든 직업은/는?', '응답': '1. 건설 노동자\n2. 해병대 솔저\n3. 농부\n4. 석유 시추기\n5. 철강작업자\n6. 어부\n7. 화물 운송 업무\n8. 석재 공사자\n9. 재활용 업무\n10. 병변 한의사'}


In [41]:
print(get_answer('직업', '정신적으로 힘든 직업'))

{'질의': '직업중에서 정신적으로 힘든 직업은/는?', '응답': '의사, 간호사, 소방관, 경찰관, 군인, 사회복지사 등이 정신적으로 힘든 직업으로 여겨질 수 있습니다. 이들 직업에서는 업무의 복잡성, 책임의 크기, 고도의 스트레스 상황에서의 대처 등이 정신적으로 많은 힘을 요구하기 때문입니다.'}


In [40]:
print(get_answer('직업', '연봉이 가장 높은 직업'))

{'질의': '직업중에서 연봉이 가장 높은 직업은/는?', '응답': '의사, 변호사, CEO, 투자은행원, 소프트웨어 엔지니어 등 다양한 직업 중에서 연봉이 가장 높은 직업은 CEO (Chief Executive Officer, 최고경영자)입니다. CEO는 기업의 최고경영자로서 회사의 전반적인 경영을 총괄하고 이끄는 역할을 맡고 있으며, 이에 따라 높은 연봉을 받는 경우가 많습니다. 그러나 실제로 연봉이 가장 높은 직업은 다양한 요인에 따라 다를 수 있으므로 일반화하기는 어렵습니다.'}


In [39]:
print(get_answer('직업', '만족도가 가장 높은 직업'))

{'질의': '직업중에서 만족도가 가장 높은 직업은/는?', '응답': '개인마다 만족도가 높은 직업은 다를 수 있지만, 일반적으로는 자신의 관심과 능력에 맞는 직업을 수행할 때 만족도가 높을 것입니다. 또한, 자신의 노력과 열정이 직접적인 성과로 나타나는 직업이나 타인을 도와주는 일을 하는 직업 등이 만족도가 높을 수 있습니다. 결국, 가장 만족도가 높은 직업은 자기 자신이 얼마나 직업에 충실하고 행복하게 일을 하는지, 타인에게 얼마나 도움을 주는지에 따라 달라질 것입니다.'}


In [45]:
question = '방금 너가 응답한 직업의 종류들을 다시 모두 알려줄래?'

answer= chat_model.invoke(question)

print('질의:', question)
print('응답:', answer.content)

질의: 방금 너가 응답한 직업의 종류들을 다시 모두 알려줄래?
응답: 네, 방금 언급된 직업들은 아래와 같습니다:
1. 프로그래머
2. 디자이너
3. 의사
4. 변호사
5. 선생님
6. 회계사
7. 간호사
8. 기자
9. 건축가
10. 여행사

더 궁금한 것이 있으시면 언제든지 물어보세요!
